<h1 style="background-color:#239B56;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
✍ תהליך הסבת שכבת גבולות תכניות לצרכי רישום בשטח לא מוסדר לבנק"ל מודרני ✍ 

In [82]:
print('16. Unregistered Tazar borders:')

16. Unregistered Tazar borders:


<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [83]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [84]:
print('  16.a Reading inputs')

  16.a Reading inputs


In [85]:
RawData_folder = r'\\mapi_shares\MNCDB\צוות מידע\RawData/'

In [86]:
today =  dt.date.today()
date_name =  str(today.year) + str(today.month) + str(today.day) + '_ncdb.gdb'
product_path = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים/' + date_name

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תצ"ר בשטח לא מוסדר

In [87]:
unreg_tazar_cols = ['TALAR_NUM', 'TALAR_YEAR', 'SURVEYOR_ID', 'GUSH_NUM', 'GUSH_SUFFIX', 'SURVEY_DATE', 'geometry']

unreg_tazar_layer = gpd.read_file(RawData_folder + 'CADSDE/TAZAR_BORDER_LOMUSDAR.gdb',
                                  layer = 'TAZAR_BORDER_LOMUSDAR',
                                  include_fields = unreg_tazar_cols)

<h1 style="text-align:right;font-size:100%">
שכבה מקומית של גבולות תצ"ר ללא מקור, בשטח לא מוסדר

In [88]:
UnkownUnregiteredazarBorders = gpd.read_file(RawData_folder + 'Local/UnkownUnregiteredazarBorders.gpkg',
                                             layer = 'UnkownUnregiteredazarBorders')

<h1 style="text-align:right;font-size:100%">
שכבת גבולות גושי קדסטר מודרניים

In [89]:
Blocks_cols = ['BlockNumber', 'SubBlockNumber', 'BlockUniqueID']

Blocks = gpd.read_file(product_path,
                       layer = 'Blocks',
                       include_fields = Blocks_cols,
                       ignore_geometry = True)

Blocks = Blocks.astype(int)


del[unreg_tazar_cols, Blocks_cols, RawData_folder, today, date_name, product_path]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [90]:
print('  16.b Cleaning data')

  16.b Cleaning data


In [91]:
unreg_tazar_layer = unreg_tazar_layer.append(UnkownUnregiteredazarBorders)
unreg_tazar_layer.reset_index(inplace=True)

<h1 style="text-align:right; font-size:100%">
הוספת שדה מקשר של מספר אליפסה מלא לצורך מיזוג

In [92]:
unreg_tazar_layer.dropna(subset=['TALAR_NUM', 'TALAR_YEAR'], inplace=True)


unreg_tazar_layer[['TALAR_NUM', 'TALAR_YEAR']] = unreg_tazar_layer[['TALAR_NUM', 'TALAR_YEAR']].astype('Int32')


bad_years_cond = unreg_tazar_layer['TALAR_YEAR'] < 1000
unreg_tazar_layer.loc[bad_years_cond, "TALAR_YEAR"] = unreg_tazar_layer['TALAR_YEAR'] + 2000
del bad_years_cond


unreg_tazar_layer['ProcessName'] = unreg_tazar_layer['TALAR_NUM'].astype('str') + '/' + unreg_tazar_layer['TALAR_YEAR'].astype('str')


unreg_tazar_layer.drop(columns=['TALAR_NUM', 'index'], inplace=True)

<h1 style="text-align:right; font-size:100%">
הוספת שדה מקשר של מספר גוש מלא לצורך מיזוג

In [93]:
unreg_tazar_layer['GUSH_SUFFIX'] = unreg_tazar_layer['GUSH_SUFFIX'].fillna(0)
unreg_tazar_layer[['GUSH_SUFFIX', 'GUSH_NUM']] = unreg_tazar_layer[['GUSH_SUFFIX', 'GUSH_NUM']].astype('Int32')
unreg_tazar_layer['BlockName'] = unreg_tazar_layer['GUSH_NUM'].astype(str) + '_' + unreg_tazar_layer['GUSH_SUFFIX'].astype(str)
unreg_tazar_layer.drop(columns=['GUSH_SUFFIX', 'GUSH_NUM'], inplace=True)

Blocks['BlockName'] = Blocks['BlockNumber'].astype(str) + '_' + Blocks['SubBlockNumber'].astype(str)

<h1 style="text-align:right; font-size:100%">
השלמת ערכים חסרים בזיהוי מודד

In [94]:
unreg_tazar_layer['SURVEYOR_ID'] = unreg_tazar_layer['SURVEYOR_ID'].fillna(0)
unreg_tazar_layer['SURVEYOR_ID'] = unreg_tazar_layer['SURVEYOR_ID'].astype('Int32')

<h1 style="text-align:right; font-size:100%">
השלמת ערכים חסרים בתאריך המדידה והפקת שנת המדידה

In [95]:
unreg_tazar_layer['SURVEY_DATE'] = pd.to_datetime(unreg_tazar_layer['SURVEY_DATE'])
unreg_tazar_layer['SurveyYear'] = unreg_tazar_layer['SURVEY_DATE'].dt.year
unreg_tazar_layer['SurveyYear'] = unreg_tazar_layer['SurveyYear'].fillna(unreg_tazar_layer['TALAR_YEAR'])
unreg_tazar_layer['SurveyYear'] = unreg_tazar_layer['SurveyYear'] .astype('Int32')


unreg_tazar_layer.drop(columns=['SURVEY_DATE', 'TALAR_YEAR'], inplace=True)

<h1 style="text-align:right; font-size:100%">
הסרת רשומות כפולות לפי מספר אליפסה וגיאומטריה זהים

In [96]:
unreg_tazar_layer = unreg_tazar_layer[~unreg_tazar_layer.duplicated(subset=['ProcessName', 'geometry'], keep='first')].sort_values('ProcessName')
unreg_tazar_layer = unreg_tazar_layer.drop_duplicates(subset=['ProcessName', 'SURVEYOR_ID'], keep='first')

In [97]:
print('  4.c Data conversion')

  4.c Data conversion


<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [98]:
print('  4.d Defining new Layer')

  4.d Defining new Layer


In [99]:
new_columns = np.dtype([
                        ("CPBUniqueID", int),
                        #("BlockUniqueID", int), יאוחד בהמשך התהליך
                        ("ProcessType", int),
                        ("MapYear", int),
                        ("ProcessName", str),
                        ("GeodeticNetwork", int),
                        ("Status", int),
                        ("PlanName", str),
                        ("SurveyorLicenseID", int),
                        ("DataSource", int),
                        ("geometry", gpd.array.GeometryDtype)
                        ])

unreg_tazar_layer_new = gpd.GeoDataFrame(np.empty(0, dtype=new_columns),
                                         geometry = 'geometry',
                                         crs = 2039)

del new_columns

<h1 style="text-align:right; font-size:100%">
איכלוס שדות

In [100]:
unreg_tazar_layer_new['geometry'] = unreg_tazar_layer['geometry']


unreg_tazar_layer_new['CPBUniqueID'] = range(1, len(unreg_tazar_layer)+1)
unreg_tazar_layer_new['CPBUniqueID'] = unreg_tazar_layer_new['CPBUniqueID'] + 11000000
unreg_tazar_layer_new['CPBUniqueID'] = unreg_tazar_layer_new['CPBUniqueID'].astype('Int64')

unreg_tazar_layer_new['MapYear'] = unreg_tazar_layer['SurveyYear'].astype('Int64')


unreg_tazar_layer_new['ProcessName'] = unreg_tazar_layer['ProcessName'].astype('str')


unreg_tazar_layer_new['ProcessType'] = 11  #תצ"ר בשטח לא מוסדר



CAD_cond = unreg_tazar_layer['SurveyYear'] <= 2015
unreg_tazar_layer_new['DataSource'] = np.where(CAD_cond, 5, 4)  # CAD source:5, SRV source:4
unreg_tazar_layer_new['DataSource'] = unreg_tazar_layer_new['DataSource'].astype('Int64')


unreg_tazar_layer_new['Status'] = 0 # unknown, will be Null in product gdb


geodeticNetDict = {range(0, 1998)    : 1,      # רשת ישראל הישנה
                   range(1999, 2007) : 2,      # רשת ישראל החדשה
                   range(2008, 99999): 3}      # רשת ישראל התקפה
unreg_tazar_layer_new['GeodeticNetwork'] = unreg_tazar_layer['SurveyYear'].apply(lambda x: next((v for k, v in geodeticNetDict.items() if x in k), 0))
unreg_tazar_layer_new['GeodeticNetwork']  = unreg_tazar_layer_new['GeodeticNetwork'].astype('Int64')


unreg_tazar_layer_new['SurveyorLicenseID'] = unreg_tazar_layer['SURVEYOR_ID'].fillna(0)
unreg_tazar_layer_new['SurveyorLicenseID'] = unreg_tazar_layer_new['SurveyorLicenseID'].astype('Int64')


unreg_tazar_layer_new['PlanName'] = None


unreg_tazar_layer_new['BlockName'] = unreg_tazar_layer['BlockName']
unreg_tazar_layer_new = unreg_tazar_layer_new.merge(Blocks, on = 'BlockName', how = 'left')
unreg_tazar_layer_new['BlockUniqueID'] = unreg_tazar_layer_new['BlockUniqueID'].fillna(0)
unreg_tazar_layer_new['BlockUniqueID'] = unreg_tazar_layer_new['BlockUniqueID'].astype('Int64')
unreg_tazar_layer_new.drop(columns=['BlockName', 'BlockNumber', 'SubBlockNumber'], inplace=True)


del unreg_tazar_layer

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  16.e Exporting results to product folder')

In [ ]:
#csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\UnregisteredTazar.csv'
json_dir =  r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\UnregisteredTazar.geojson'


#unreg_tazar_layer_new.to_csv(csv_dir, index=False)
unreg_tazar_layer_new.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([r'M:\ncdbScripts\scripts\arcpyPro\UnregisteredTazar-Save as Feature Class.bat']);